In [4]:
# 导入库
import svgwrite
import math

In [5]:
def create_box_template(output_filename="custom_box_template.svg",
                        length=300,  # 主体中间矩形宽度 (替换为实际值)
                        width=200,   # 主体中间矩形高度 (替换为实际值)
                        height=80,  # 侧边矩形高度 (替换为实际值)
                        flap_length_top_bottom=100, # 顶部和底部盖片长度 (替换为实际值)
                        flap_length_side=90,      # 侧边盖片长度 (替换为实际值)
                        top_flap_radius=30,      # 顶部盖片圆弧半径 (替换为实际值)
                        tab_width=30,             # 侧边插舌宽度 (替换为实际值)
                        tab_height=20,            # 侧边插舌高度 (替换为实际值)
                        notch_width=10,           # 主体切口宽度 (替换为实际值)
                        notch_height=40):         # 主体切口高度 (替换为实际值)
    """
    根据提供的参数创建自定义包装盒的刀板图 SVG 文件。
    """

    dwg = svgwrite.Drawing(output_filename, profile='tiny')
    factor = 1  # 比例因子

    length *= factor
    width *= factor
    height *= factor
    flap_length_top_bottom *= factor
    flap_length_side *= factor
    top_flap_radius *= factor
    tab_width *= factor
    tab_height *= factor
    notch_width *= factor
    notch_height *= factor

    # 绘制主体中间矩形
    dwg.add(dwg.rect((0, height), (length, width), fill='none', stroke='black'))

    # 绘制顶部盖片 (带圆弧)
    top_flap_path = dwg.path(fill='none', stroke='black')
    top_flap_path.push(f"M {0} {height}")
    top_flap_path.push(f"v -{flap_length_top_bottom - top_flap_radius}")
    top_flap_path.push(f"a {top_flap_radius},{top_flap_radius} 0 0 1 {top_flap_radius - top_flap_radius * math.cos(math.pi / 4)},{-top_flap_radius + top_flap_radius * math.sin(math.pi / 4)}") # 简化圆弧
    top_flap_path.push(f"h {length - 2 * (top_flap_radius - top_flap_radius * math.cos(math.pi / 4))}")
    top_flap_path.push(f"a {top_flap_radius},{top_flap_radius} 0 0 1 {top_flap_radius - top_flap_radius * math.cos(math.pi / 4)},{top_flap_radius - top_flap_radius * math.sin(math.pi / 4)}")
    top_flap_path.push(f"v {flap_length_top_bottom - top_flap_radius}")
    dwg.add(top_flap_path)

    # 绘制底部盖片 (带圆弧 - 镜像)
    bottom_flap_path = dwg.path(fill='none', stroke='black')
    bottom_flap_path.push(f"M {0} {height + width}")
    bottom_flap_path.push(f"v {flap_length_top_bottom - top_flap_radius}")
    bottom_flap_path.push(f"a {top_flap_radius},{top_flap_radius} 0 0 0 {top_flap_radius - top_flap_radius * math.cos(math.pi / 4)},{top_flap_radius - top_flap_radius * math.sin(math.pi / 4)}")
    bottom_flap_path.push(f"h {length - 2 * (top_flap_radius - top_flap_radius * math.cos(math.pi / 4))}")
    bottom_flap_path.push(f"a {top_flap_radius},{top_flap_radius} 0 0 0 {top_flap_radius - top_flap_radius * math.cos(math.pi / 4)},{-top_flap_radius + top_flap_radius * math.sin(math.pi / 4)}")
    bottom_flap_path.push(f"v -{flap_length_top_bottom - top_flap_radius}")
    dwg.add(bottom_flap_path)

    # 绘制左侧边矩形和盖片
    dwg.add(dwg.rect((-height, height), (height, width), fill='none', stroke='black'))
    dwg.add(dwg.rect((-height - flap_length_side, height), (flap_length_side, width), fill='none', stroke='black'))
    dwg.add(dwg.rect((-height, height + width - tab_height - 2), (tab_width, tab_height), fill='none', stroke='black')) # 上插舌 (粗略位置)
    dwg.add(dwg.rect((-height, height + tab_height + 2), (tab_width, tab_height), fill='none', stroke='black')) # 下插舌 (粗略位置)

    # 绘制右侧边矩形和盖片 (镜像)
    dwg.add(dwg.rect((length, height), (height, width), fill='none', stroke='black'))
    dwg.add(dwg.rect((length + height, height), (flap_length_side, width), fill='none', stroke='black'))
    dwg.add(dwg.rect((length, height + width - tab_height - 2), (tab_width, tab_height), fill='none', stroke='black')) # 上插舌 (粗略位置)
    dwg.add(dwg.rect((length, height + tab_height + 2), (tab_width, tab_height), fill='none', stroke='black')) # 下插舌 (粗略位置)

    # 绘制主体上的切口 (粗略位置)
    dwg.add(dwg.rect((0, height + (width - 2 * notch_height) / 3), (notch_width, notch_height), fill='none', stroke='black')) # 左侧上切口
    dwg.add(dwg.rect((0, height + 2 * (width - 2 * notch_height) / 3 + notch_height), (notch_width, notch_height), fill='none', stroke='black')) # 左侧下切口
    dwg.add(dwg.rect((length - notch_width, height + (width - 2 * notch_height) / 3), (notch_width, notch_height), fill='none', stroke='black')) # 右侧上切口
    dwg.add(dwg.rect((length - notch_width, height + 2 * (width - 2 * notch_height) / 3 + notch_height), (notch_width, notch_height), fill='none', stroke='black')) # 右侧下切口

    # 添加折叠线 (虚线) - 需要更精确的定位
    fold_lines = [
        ((0, height), (length, height)),          # 上主体折叠
        ((0, height + width), (length, height + width)), # 下主体折叠
        ((0, height), (0, height + width)),      # 左主体折叠
        ((length, height), (length, height + width)), # 右主体折叠

        ((0, height - flap_length_top_bottom), (0, height)),      # 顶部盖片折叠
        ((length, height - flap_length_top_bottom), (length, height)), # 顶部盖片折叠
        ((0, height + width + flap_length_top_bottom), (0, height + width)), # 底部盖片折叠
        ((length, height + width + flap_length_top_bottom), (length, height + width)), # 底部盖片折叠

        ((-height, height), (0, height)),        # 左侧边折叠
        ((-height, height + width), (0, height + width)),    # 左侧边折叠
        ((length + height, height), (length, height)),      # 右侧边折叠
        ((length + height, height + width), (length, height + width)),  # 右侧边折叠

        ((-height - flap_length_side, height), (-height, height)), # 左侧盖片折叠
        ((-height - flap_length_side, height + width), (-height, height + width)), # 左侧盖片折叠
        ((length + height + flap_length_side, height), (length + height, height)), # 右侧盖片折叠
        ((length + height + flap_length_side, height + width), (length + height, height + width)), # 右侧盖片折叠
    ]
    for start, end in fold_lines:
        dwg.add(dwg.line(start, end, stroke='gray', stroke_dasharray="5,5"))

    dwg.save()
    print(f"刀板图已保存为 {output_filename}")

In [6]:
if True:
    # 替换为实际测量的数值！
    box_length = 220  # 替换
    box_width = 330   # 替换
    box_height = 65    # 替换
    top_bottom_flap = 65 # 替换
    side_flap = 65      # 替换
    top_radius = 20     # 替换
    side_tab_width = 20  # 替换
    side_tab_height = 15 # 替换
    main_notch_width = 5  # 替换
    main_notch_height = 20 # 替换

    create_box_template(
        output_filename="cutie_custom_box.svg",
        length=box_length,
        width=box_width,
        height=box_height,
        flap_length_top_bottom=top_bottom_flap,
        flap_length_side=side_flap,
        top_flap_radius=top_radius,
        tab_width=side_tab_width,
        tab_height=side_tab_height,
        notch_width=main_notch_width,
        notch_height=main_notch_height
    )

刀板图已保存为 cutie_custom_box.svg
